# LOB Skeleton

Generic, theory-focused skeleton for discrete event simulation w poisson arrivals and 2 interacting queues

---

## Steps

##### Minimal Adaptation Guide (Discrete-Event Simulation → Limit Order Book)

##### 1. Replace generic queues
- Queue A → **Bid book** (buy limit orders), sorted by *(price desc, time)*  
- Queue B → **Ask book** (sell limit orders), sorted by *(price asc, time)*  

##### 2. Replace event types with market events
Use independent Poisson processes for:
- `limit_buy_arrival`
- `limit_sell_arrival`
- `market_buy_arrival`
- `market_sell_arrival`
- `cancel`

##### 3. Limit order arrival handler
- Create new order with fields: `side`, `price`, `time_created`, `owner`
- Insert into appropriate book (bid/ask)
- Immediately attempt to match against opposite side (price crossing rule)

##### 4. Market order arrival handler
- Consume the best-priced order on the opposite book
- Record:
  - trade price  
  - execution time  
  - market-maker P/L effects  

##### 5. Price-time priority
Maintain ordering:
- Bids: sorted by **highest price first**, earliest time at a given price
- Asks: sorted by **lowest price first**, earliest time at a given price

This enforces the correct matching discipline.

##### 6. Market maker logic
On fixed intervals or after each event:
- Compute midprice from best bid/ask
- Adjust quotes using **inventory-based skew**
- Post 1-unit bid and ask limit orders
- Track:
  - inventory  
  - cash  
  - P/L = `cash + inventory * last_trade_price`

##### 7. Cancellation event
- Randomly remove one resting order from either book
- If it was an MM order, update MM’s internal state

##### 8. Metrics to record
- Bid–ask spread over time  
- Execution times of filled orders  
- Market-maker:
  - profit/loss  
  - inventory  
- Number of trades  
- Book depth (optional)

##### 9. Replications
Use:
```
for r in range(NUM_REPS):
    sim = ...
    results.append(sim.run())
```
Analyze how varying arrival/cancel rates and MM parameters affects:
- execution efficiency  
- liquidity  
- volatility  
- inventory risk  
- stress conditions (high arrival rates)



---

## Skeleton

In [1]:
import numpy as np
from dataclasses import dataclass
from typing import List, Optional

# =====================================================================
# Discrete-Event Simulation Skeleton
# ---------------------------------------------------------------------
# - Continuous time
# - Poisson event arrivals
# - Two interacting queues (Queue A and Queue B)
# - When both queues are non-empty, entities can be "matched" (served)
# - Includes cancellations/abandonments and basic performance measures
# 
# This is meant as a clear *template* for simulation theory:
#   - state variables
#   - event types
#   - random variate generation (Poisson/exponential)
#   - event handling
#   - performance data collection
#   - multiple replications
# =====================================================================

# ---------------------------------------------------------------------
# Data structures
# ---------------------------------------------------------------------

@dataclass
class Entity:
    """
    Generic entity in the system.

    Attributes:
        id:          Unique entity ID.
        entity_type: 'A' or 'B' (type of queue it belongs to).
        time_created: Simulation time at which the entity arrived.
    """
    id: int
    entity_type: str
    time_created: float


class TwoSidedQueueSimulation:
    """
    Discrete-event simulation of a generic two-sided system:

        - Queue A: holds entities of type A
        - Queue B: holds entities of type B

    Event types (all arrive according to independent Poisson processes):
        - arrival_A: Type A entity arrives and joins Queue A
        - arrival_B: Type B entity arrives and joins Queue B
        - cancel:    A randomly selected entity abandons (from either queue)

    Matching logic:
        - Whenever both queues are non-empty, the *next event* can be
          a "matching" event (one A + one B leave together).
        - Here we model matching as an *additional* Poisson event type.
          (This is simple and keeps event times exponential.)

    This is structurally similar to:
        - a two-sided queueing system
        - a generic "matching model" (e.g., supply vs demand)
    """

    def __init__(
        self,
        sim_duration: float,
        # Poisson rates (events per unit time)
        lam_arrival_A: float,
        lam_arrival_B: float,
        lam_cancel: float,
        lam_match: float,
        seed: Optional[int] = None,
    ):
        # Simulation horizon
        self.sim_duration = sim_duration

        # Event rates
        self.lam_arrival_A = lam_arrival_A
        self.lam_arrival_B = lam_arrival_B
        self.lam_cancel = lam_cancel
        self.lam_match = lam_match

        # RNG
        self.rng = np.random.default_rng(seed)

        # State variables
        self.time: float = 0.0
        self.queue_A: List[Entity] = []
        self.queue_B: List[Entity] = []
        self.next_entity_id: int = 0

        # Performance metrics (raw data; summary done at the end)
        self.num_matches: int = 0
        self.waiting_times_A: List[float] = []
        self.waiting_times_B: List[float] = []
        self.queue_length_record_times: List[float] = []
        self.queue_lengths_A: List[int] = []
        self.queue_lengths_B: List[int] = []

    # -----------------------------------------------------------------
    # Helper methods
    # -----------------------------------------------------------------

    def _new_entity_id(self) -> int:
        eid = self.next_entity_id
        self.next_entity_id += 1
        return eid

    def _record_queue_lengths(self):
        """
        Record the queue lengths at the current simulation time.
        This is a form of "time-dependent" performance measure.
        """
        self.queue_length_record_times.append(self.time)
        self.queue_lengths_A.append(len(self.queue_A))
        self.queue_lengths_B.append(len(self.queue_B))

    # -----------------------------------------------------------------
    # Event handlers
    # -----------------------------------------------------------------

    def _handle_arrival(self, entity_type: str):
        """
        Generic arrival event.
            - Create new entity
            - Add it to appropriate queue
        """
        e = Entity(
            id=self._new_entity_id(),
            entity_type=entity_type,
            time_created=self.time,
        )
        if entity_type == "A":
            self.queue_A.append(e)
        elif entity_type == "B":
            self.queue_B.append(e)
        else:
            raise ValueError("Unknown entity type")

    def _handle_cancellation(self):
        """
        Cancellation/abandonment event.
        Randomly removes one entity from the system (if any).
        This is just one simple modeling choice for abandonments.
        """
        total = len(self.queue_A) + len(self.queue_B)
        if total == 0:
            return  # nothing to cancel

        idx = self.rng.integers(0, total)
        if idx < len(self.queue_A):
            self.queue_A.pop(idx)
        else:
            self.queue_B.pop(idx - len(self.queue_A))

    def _handle_matching(self):
        """
        Matching event:
            - One entity from Queue A and one from Queue B leave together
            - Waiting times for both are recorded
        If either queue is empty, the event does nothing (but it still
        consumes time, because it was scheduled by the Poisson clock).
        """
        if len(self.queue_A) == 0 or len(self.queue_B) == 0:
            return

        # Here we use simple FIFO (first-come-first-served) in each queue.
        # Other priority disciplines could be used instead.
        eA = self.queue_A.pop(0)
        eB = self.queue_B.pop(0)

        self.num_matches += 1

        # Waiting time = current time - arrival time
        self.waiting_times_A.append(self.time - eA.time_created)
        self.waiting_times_B.append(self.time - eB.time_created)

    # -----------------------------------------------------------------
    # Main simulation loop
    # -----------------------------------------------------------------

    def run(self):
        """
        Core discrete-event loop.

        We treat arrivals, cancellations, and matching as *competing*
        Poisson processes. The time to the next event is exponential
        with parameter equal to the sum of all active rates.
        The event type is chosen with probability proportional to its rate.
        """
        # Initial record of queue lengths at time 0
        self._record_queue_lengths()

        # Precompute constant event name list
        event_names = ["arrival_A", "arrival_B", "cancel", "match"]

        while self.time < self.sim_duration:
            # For clarity, recompute total rate each step.
            # In more complex models, some rates might depend on state.
            rates = np.array([
                self.lam_arrival_A,
                self.lam_arrival_B,
                self.lam_cancel,
                self.lam_match,
            ], dtype=float)

            total_rate = rates.sum()
            if total_rate <= 0:
                # No more events can occur
                break

            # Time advance: exponential(1 / total_rate)
            dt = self.rng.exponential(1.0 / total_rate)
            self.time += dt

            # Choose event type according to probabilities = rate / total_rate
            probs = rates / total_rate
            event = self.rng.choice(event_names, p=probs)

            # Handle the chosen event
            if event == "arrival_A":
                self._handle_arrival("A")
            elif event == "arrival_B":
                self._handle_arrival("B")
            elif event == "cancel":
                self._handle_cancellation()
            elif event == "match":
                self._handle_matching()
            else:
                raise ValueError("Unknown event type")

            # After each event, we can record time-dependent statistics
            self._record_queue_lengths()

        # At the end, we may compute summary statistics
        avg_wait_A = np.mean(self.waiting_times_A) if self.waiting_times_A else np.nan
        avg_wait_B = np.mean(self.waiting_times_B) if self.waiting_times_B else np.nan

        return {
            "num_matches": self.num_matches,
            "avg_wait_A": avg_wait_A,
            "avg_wait_B": avg_wait_B,
            "queue_length_times": self.queue_length_record_times,
            "queue_lengths_A": self.queue_lengths_A,
            "queue_lengths_B": self.queue_lengths_B,
        }


# ---------------------------------------------------------------------
# Multiple replications driver (for statistical analysis)
# ---------------------------------------------------------------------

if __name__ == "__main__":
    NUM_REPS = 5
    SIM_DURATION = 100.0  # time units

    # Parameters for all replications (you can vary across reps instead)
    lam_arrival_A = 0.8
    lam_arrival_B = 0.8
    lam_cancel = 0.2
    lam_match = 1.0

    all_results = []
    for r in range(NUM_REPS):
        sim = TwoSidedQueueSimulation(
            sim_duration=SIM_DURATION,
            lam_arrival_A=lam_arrival_A,
            lam_arrival_B=lam_arrival_B,
            lam_cancel=lam_cancel,
            lam_match=lam_match,
            seed=123 + r,  # different stream each replication
        )
        res = sim.run()
        all_results.append(res)
        print(
            f"Replication {r}: "
            f"matches={res['num_matches']}, "
            f"avg_wait_A={res['avg_wait_A']:.3f}, "
            f"avg_wait_B={res['avg_wait_B']:.3f}"
        )

    # All per-replication data are now in `all_results` for further analysis


Replication 0: matches=72, avg_wait_A=8.221, avg_wait_B=5.065
Replication 1: matches=67, avg_wait_A=3.232, avg_wait_B=5.995
Replication 2: matches=59, avg_wait_A=2.652, avg_wait_B=2.997
Replication 3: matches=67, avg_wait_A=3.432, avg_wait_B=3.616
Replication 4: matches=63, avg_wait_A=4.335, avg_wait_B=4.348


---

## Notes

Andrew Test